In [1]:
import pandas as pd
import numpy as np
import uuid
import _pickle as pickle
import os
import configparser
import pymysql
import multiprocessing as mp
import sys
import uuid
from functools import reduce
from imp import reload
from Code import data_generator_new
from scipy.sparse import coo_matrix
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

In [ ]:
reload(data_generator_new)

### 数据探查

In [2]:
config = configparser.ConfigParser()
config.read("../Data/Input/database_config/database.conf")
host = config['ASSESSMENT']['host']
user = config['ASSESSMENT']['user']
password = config['ASSESSMENT']['password']
database = config['ASSESSMENT']['database']
port = config['ASSESSMENT']['port']
charset = config['ASSESSMENT']['charset']
db = pymysql.connect(host=host, user=user, password=password, db=database, port=int(port), charset=charset)
config.read("../Data/Input/Tag_graph/filter.conf")
ctag_keep_list = config['FILTER']['filter_list'].split(",")

In [3]:
new_result = "company_tag_info_latest0703"
old_result = "company_online_tag_info_latest"
main_tags = "comp_main_business_0726"

In [4]:
sql_new_result = "select comp_id, comp_full_name, label_name, classify_id, label_type, label_type_num, one_src_tags from %s" % new_result
sql_old_result = "select comp_id, comp_full_name, label_name, main_business_type as 1.0 from %s" % old_result
sql_main_tags = "select comp_id, comp_full_name, info_extra_tag, name_tag from %s" % main_tags

In [5]:
main_tags = pd.read_sql(sql_main_tags, con=db)

In [6]:
main_tags.fillna("", inplace=True)

In [8]:
main_tags.head(3)

,comp_id,comp_full_name,info_extra_tag,name_tag
0,10000030532042217410,内蒙古引绰济辽供水有限责任公司,运营管理-建设-投资-工程\n,
1,10000030535836499710,武汉鹊人医疗科技服务有限公司,销售-Ⅱ类-医疗器械-安装\n,医疗科技-医疗科技服务
2,1000003426482140130,北京华夏中青社区服务有限公司第一分公司,,社区服务


In [9]:
main_tags.info_extra_tag = main_tags.info_extra_tag.apply(lambda x: reduce(lambda m, n: m + n, x.strip().split("&")))

In [10]:
main_tags["tag_list"] = main_tags[["comp_id", "info_extra_tag", "name_tag"]] \
    .apply(lambda x: [(x[0], a, 1.0) for l in x[1].strip().split("&") for m in l.split("#") for a in m.split("-")] +  \
           [(x[0], a, 2.0) for l in x[2].strip().split("&") for n in l.split("#") for a in n.split("-")],  \
           axis=1)

In [11]:
main_tags_flatted = pd.DataFrame([t for y in main_tags.tag_list for t in y], columns=["comp_id", "label_name", "main_type"])

In [15]:
pickle.dump(main_tags_flatted, open("main_tags_flatted.pkl", "wb"))

### 普通标签和主营标签融合

In [ ]:
comp_tag = pickle.load(open("../Data/Output/Tag_graph/v1/comp_tag.pkl", "rb"))

In [ ]:
main_tags_flatted = pickle.load(open("main_tags_flatted.pkl", "rb"))

In [ ]:
merged = comp_tag.merge(main_tags_flatted, on=["comp_id", "label_name"], how="outer")

In [ ]:
len(merged)

In [9]:
pickle.dump(merged, open("comp_tag_merged.pkl", "wb"))

### 命中统计

In [2]:
merged = pickle.load(open("comp_tag_merged.pkl", "rb"))

In [3]:
word_count = merged.label_name.value_counts().to_frame()

In [4]:
word_count.index.name = "label_name"
word_count.rename(columns={"label_name": "label_count"}, inplace=True)
word_count.reset_index(inplace=True)

In [5]:
word_count[word_count.label_count > 10].count()

label_name     64412
label_count    64412
dtype: int64

In [6]:
word_count.sort_values(by="label_count", ascending=False)

,label_name,label_count
0,销售,596265
1,商务服务,490793
2,生产,254659
3,房产家装,248687
4,其他商务服务业,220498
5,新型材料产业,213873
6,房地产开发,196200
7,五金交电,188811
8,加工,184362
9,企业,179313


### 标签和公司编码

In [9]:
merged.fillna({"type": 1.0, "count_comps": 1.0, "main_type": 0.0}, inplace=True)

In [8]:
sum(merged.type.isnull())

24752500

In [17]:
merged.head(3)

,comp_id,label_name,type,count_comps,tag_uuid,main_type,comp_int_id
0,10000030532042217410,农业电子商务,0.0,1.0,0c7c86960dcc575b9a9b9831ed1aeaad,0.0,0
1,10000030532042217410,农产品生产,0.0,1.0,06af12bdfd38563ea6dd653ea85dc7c7,0.0,0
2,10000030532042217410,渔业生产,0.0,1.0,3083285f99d850a6b4fdce8207ccd537,0.0,0


In [12]:
merged.drop(["comp_int_id"], axis=1, inplace=True)

In [13]:
comp_id_dict = merged["comp_id"].drop_duplicates().reset_index(drop=True)
comp_id_dict = comp_id_dict.reset_index()
comp_id_dict.rename(index=str, columns={"index": "comp_int_id"}, inplace=True)

In [16]:
merged = merged.merge(comp_id_dict, how="left", left_on="comp_id", right_on="comp_id")

In [18]:
nctag_filter_num=(10, 1000000)

In [ ]:
tag_comps_aggregated = merged.groupby("label_name") \
    .agg({"comp_int_id": lambda x: set(x), "count_comps": "count", "type": lambda x: x[0], "main_type": lambda x: x[0]}).reset_index()

In [ ]:
tag_comps_aggregated = tag_comps_aggregated[((tag_comps_aggregated.count_comps > nctag_filter_num[0]) \
                                             & (tag_comps_aggregated.count_comps < nctag_filter_num[1])) \
                                             | (tag_comps_aggregated.main != 0)]\
                                             [["label_name", "comp_int_id", "type"]].reset_index(drop=True)

### 检测comp_tag函数

In [ ]:
data_generator_new.comp_tag()

### 检测data_aggregator函数，返回概念、非概念各自聚合公司的结果，以及公司标签关联、公司总数

In [ ]:
data_generator_new.data_aggregator()

### 计算公司标签全集

In [ ]:
data_generator_new.properties()

### 计算标签关联

In [ ]:
data_generator_new.tag_tag4(batch_size=4, process_num=8)

### 处理成导入mysql和neo4j的文件

In [ ]:
data_generator_new.neo4j_merge()

### 导入图数据库

In [ ]:
data_generator_new.to_graph_database()

In [ ]:
spark

In [ ]:
sc

### 全流程测试

In [ ]:
!python Code/pipline_new.py

In [ ]:
1

### 智能网联汽车关联的top1000概念非概念标签

In [ ]:
relations = pd.read_csv("../Data/Output/Tag_graph/all_relations.csv", header=None)

In [ ]:
relations.columns = ["src_id", "target_id", "rel_value", "rel_type"]

In [ ]:
relations.head(1)

In [ ]:
tag_dict = pickle.load(open("../Data/Output/Tag_graph/tag_dict.pkl", "rb"))
tag_dict_reverse = dict(zip(tag_dict.values(), tag_dict.keys()))

In [ ]:
target_tag = tag_dict.get("智能网联汽车")

In [ ]:
target_tag

In [ ]:
linked_ctag_top_1000 = relations[(relations.src_id == target_tag) & (relations.rel_type == "ctag_ctag")].sort_values(by="rel_value", ascending=False)

In [ ]:
len(linked_ctag_top_1000)

In [ ]:
_linked_ctag_top_1000 = relations[(relations.src_id == target_tag) & (relations.rel_type == "ctag_ctag")].sort_values(by="rel_value", ascending=False)

In [ ]:
len(_linked_ctag_top_1000)

In [ ]:
linked_nctag_top_1000 = relations[(relations.target_id == target_tag) & (relations.rel_type == "ctag_ctag")].sort_values(by="rel_value", ascending=False)[:1000]

In [ ]:
len(linked_nctag_top_1000)

In [ ]:
_linked_nctag_top_1000 = relations[(relations.src_id == target_tag) & (relations.rel_type == "ctag_ntag")].sort_values(by="rel_value", ascending=False)[:1000]

In [ ]:
linked_ctag_top_1000["target_tag_name"] = linked_ctag_top_1000.targe_id.apply(lambda x: tag_dict_reverse.get(x))[:1000]

In [ ]:
linked_nctag_top_1000["target_tag_name"] = linked_nctag_top_1000.targe_id.apply(lambda x: tag_dict_reverse.get(x))[:1000]